Welcome to the inference notebook for the Urban Sound Classification project (NAML-2024 project by Lorenzo Gentile).
This notebook lets you load trained models to make predictions on the test set. You can also plot the confusion matrix and inspect the feature of intermediate layers of the model.

In [ ]:
import tensorflow as tf
tf.keras.utils.set_random_seed(42)

Here we load one of the available trained models from the SAVED_MODELS_DIR directory. 
The notebook will prompt you to select the model you want to load. Along with the model, also the parameters used for loading and preprocessing the data and those used for training the model will be loaded. In this way, you can reproduce the same preprocessing steps and use a test set that is consistent with the one used for training (the model has not trained on samples from the test set).

In [ ]:
SAVED_MODELS_DIR = 'saved_models/'

from scripts.model_utils import list_and_load_model

# Load a model
model, preprocess_params, training_params = list_and_load_model(SAVED_MODELS_DIR)

In this cell we unpack and print the preprocessing parameters and the training parameters, which were saved in JSON format in the assets directory of the model.

In [ ]:
from pprint import pprint

print("Preprocess parameters:")
pprint(preprocess_params, sort_dicts=False)

print("\nTraining parameters:")
pprint(training_params, sort_dicts=False)

In the next cell, we load the dataset on which the model was trained and we apply the exact same preprocessing steps. Then we split the dataset to obtain the test set (which thanks to the reproducibility of the preprocessing steps is consistent with the one used for training).

In [ ]:
from scripts.dataset_loading import load_dataset
from scripts.audio_preprocessing import apply_resizing, transform_normalize_dataset

# Load and process audio files with same configuration

batch_size = training_params['BATCH_SIZE']

_, test_ds, class_names = load_dataset(preprocess_params)

if preprocess_params['DATASET_NAME'] == 'UrbanSound8K':
    test_ds = apply_resizing(test_ds, preprocess_params)
test_ds = transform_normalize_dataset(test_ds, preprocess_params)
test_ds = test_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

In this cell, you can plot the confusion matrix of the model on the test set.

In [ ]:
from scripts.model_utils import print_confusion_matrix

# Print the confusion matrix
print_confusion_matrix(model, test_ds, class_names)

In this cell, you can inspect the feature maps of the intermediate layers of the model. You can select the layer you want to inspect from the dropdown menu. You can additionally pass a seed as last argument to change the input sample used for the visualization.

In [ ]:
from scripts.model_utils import plot_feature_maps

plot_feature_maps(model, test_ds, class_names, 100)

Finally, in this cell, you can visualize the filters of the convolutional layers of the model. Filters of all convolutional layers are displayed.

In [ ]:
from scripts.model_utils import plot_conv_weights

# Visualize the convolutional weights
plot_conv_weights(model)